# Operations Reasearch: Case 2 Problem 2

In [1]:
from gurobi import *
import pandas as pd

In [2]:
case_2 = Model("case_2")

Academic license - for non-commercial use only - expires 2021-06-02
Using license file /Users/sunny/gurobi.lic


## Parameters

In [3]:
S_I = 100 # Set of products
S_J = 3   # Set of shipping methods
S_T = 26  # Set of months
S_V = 2

In [4]:
url = r'OR109-2_case02_data_s5.xlsx'

# D_it
Demand_info = pd.read_excel(url, 'Demand').iloc[:,1:]
Demand = []
for i in Demand_info.index:
    Demand.append(list(Demand_info.loc[i]))

# I_it
Init_inv_info = pd.read_excel(url, 'Initial inventory')
In_transit_info = pd.read_excel(url, 'In-transit').iloc[:,2:]
In_transit = []
for i in In_transit_info.index:
    In_transit.append([list(Init_inv_info['Initial inventory'])[i]] + list(In_transit_info.loc[i]) + [0] * (S_T - 3))

# C^V_ij
Shipping_cost_info = pd.read_excel(url, 'Shipping cost').iloc[:,1:]
Shipping_cost = []
for i in Shipping_cost_info.index:
    Shipping_cost.append(list(Shipping_cost_info.loc[i]) + [0])
print(Shipping_cost)
Price_and_cost_info = pd.read_excel(url, 'Price and cost').iloc[:,1:]
# C^P_i
Purchasing_cost = list(Price_and_cost_info['Purchasing cost'])
# C^H_i
Holding_cost = list(Price_and_cost_info['Holding cost'])

# C^F_j
Fixed_cost_shipping_method = [100, 80, 50]

# T_j
Lead_time = [1, 2, 3]

# V_i
Product_size_info = pd.read_excel(url, 'Size')
Product_size = list(Product_size_info['Size'])

Shortage_info = pd.read_excel(url, 'Shortage')
# C^L_i
Lost_sales_cost = list(Shortage_info['Lost sales'])
# C^B_i
Backorder_cost = list(Shortage_info['Backorder'])
# beta_i
Backorder_percentage = list(Shortage_info['Backorder percentage'])

Vendor_product_info = pd.read_excel(url, 'Vendor-Product')
Vendor_product = list(Vendor_product_info['Vendor'])

Vendor_cost_info = pd.read_excel(url, 'Vendor cost')
Vendor_cost = list(Vendor_cost_info['Ordering cost'])

Bounds_info = pd.read_excel(url, 'Bounds')
Bounds = list(Bounds_info['Minimum order quantity (if an order is placed)'])

Conflict_info = pd.read_excel(url, 'Conflict')
product1_list = list(Conflict_info['Product 1'])
product2_list = list(Conflict_info['Product 2'])
Conflict = []
for i in range(len(product1_list)):
    Conflict.append(tuple([(int(product1_list[i]) - 1), (int(product2_list[i])- 1)]))

# variable shipping cost
ocean_shipping_cost = []
for i in range(len(Product_size)):
    unit_cost = Product_size[i] / 0.5 * 1500
    ocean_shipping_cost.append(unit_cost)

product_demand = []
product_demand_from4 = []
for i in range(len(Demand)):    
    product_demand.append(sum(Demand[i]))
    product_demand_from4.append(product_demand[i] - max(sum(In_transit[i]), sum(Demand[i][0:3])))


[[44, 18, 0], [89, 45, 0], [86, 38, 0], [91, 46, 0], [50, 21, 0], [51, 25, 0], [83, 46, 0], [96, 49, 0], [80, 35, 0], [49, 20, 0], [53, 37, 0], [80, 75, 0], [86, 63, 0], [90, 88, 0], [50, 36, 0], [41, 27, 0], [73, 53, 0], [92, 82, 0], [81, 49, 0], [56, 39, 0], [48, 27, 0], [84, 63, 0], [86, 61, 0], [85, 84, 0], [58, 43, 0], [46, 29, 0], [65, 40, 0], [101, 82, 0], [78, 43, 0], [47, 36, 0], [46, 39, 0], [75, 61, 0], [78, 50, 0], [86, 83, 0], [58, 51, 0], [41, 27, 0], [71, 48, 0], [106, 99, 0], [84, 61, 0], [37, 20, 0], [51, 34, 0], [80, 79, 0], [75, 60, 0], [79, 59, 0], [68, 65, 0], [36, 27, 0], [76, 43, 0], [110, 62, 0], [81, 54, 0], [38, 21, 0], [42, 22, 0], [86, 68, 0], [78, 74, 0], [84, 64, 0], [61, 56, 0], [38, 30, 0], [77, 51, 0], [118, 118, 0], [77, 49, 0], [34, 25, 0], [48, 32, 0], [83, 73, 0], [74, 72, 0], [91, 74, 0], [55, 34, 0], [30, 20, 0], [71, 44, 0], [113, 69, 0], [67, 65, 0], [31, 19, 0], [44, 26, 0], [76, 72, 0], [77, 40, 0], [98, 84, 0], [48, 28, 0], [32, 32, 0], [68, 

## Choosing shipping method

In [5]:
ocean_cost_sum_from4 = 0
for i in range(len(ocean_shipping_cost)):
    ocean_cost_sum_from4 += product_demand_from4[i]*ocean_shipping_cost[i]
ocean_cost_sum_from4 += 50 * len(Demand[0])

express_cost_sum_from4 = 0
for i in range(len(Shipping_cost)):
    express_cost_sum_from4 += product_demand_from4[i]*Shipping_cost[i][0]
express_cost_sum_from4 += 100*len(Demand[0])

air_cost_sum_from4 = 0
for i in range(len(Shipping_cost)):
    air_cost_sum_from4 += product_demand_from4[i]*Shipping_cost[i][1]
air_cost_sum_from4 += 80*len(Demand[0])

if express_cost_sum_from4 <= air_cost_sum_from4 and express_cost_sum_from4 <= ocean_cost_sum_from4:
    shipping_method = 0
elif air_cost_sum_from4 <= express_cost_sum_from4 and air_cost_sum_from4 <= ocean_cost_sum_from4:
    shipping_method = 1
else:
    shipping_method = 2
print(shipping_method)

1


# Vendors

In [6]:
Vendor_list = list(set(Vendor_product))
print(Vendor_list)

Vendor_sepr_index = []

for i in range(len(Vendor_list)):
    Vendor_sepr_index.append(Vendor_product.index(Vendor_list[i]))

Vendor_sepr_index += [len(Vendor_product)]
print(Vendor_sepr_index)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[0, 66, 124, 155, 198, 228, 259, 300, 366, 410, 442, 507, 548, 600]


## M

In [7]:
M = quicksum(quicksum(Demand[i][t] for i in range(S_I))for t in range(S_T))

## Variables

In [8]:
x_vars = []  # product ordered
for i in range(S_I):
    x_vars.append([])
    for j in range(S_J):
        x_vars[i].append([])
        for t in range(S_T):
            x_vars[i][j].append(case_2.addVar(lb = 0, vtype=GRB.CONTINUOUS,
                                              name=('x' + str(i+1) + ',' + str(j+1) + ',' + str(t+1))))

u_vars = []  # inventory level at the end of period
for i in range(S_I):
    u_vars.append([])
    for t in range(S_T):
        u_vars[i].append(case_2.addVar(lb = 0, vtype=GRB.CONTINUOUS,
                                       name=('u' + str(i+1) + ',' + str(t+1))))

y_vars = []  # if method j is used
for j in range(S_J):
    y_vars.append([])
    for t in range(S_T):
        y_vars[j].append(case_2.addVar(lb = 0, vtype = GRB.BINARY,
                                       name=('y' + str(j+1) + ',' + str(t+1))))

z_vars = []  # containers needed
for t in range(S_T):
    z_vars.append(case_2.addVar(lb = 0, vtype = GRB.INTEGER,
                                        name=('z' + str(t+1))))

s_vars = []  # shortage level
for i in range(S_I):
    s_vars.append([])
    for t in range(S_T):
        s_vars[i].append(case_2.addVar(lb = 0, vtype=GRB.CONTINUOUS,
                                       name=('s' + str(i+1) + ',' + str(t+1))))

k_vars = []  # 1 if the inventory i level is 0 at the end of period t
for i in range(S_I):
    k_vars.append([])
    for t in range(S_T):
        k_vars[i].append(case_2.addVar(lb = 0, vtype=GRB.BINARY,
                                       name=('k' + str(i+1) + ',' + str(t+1))))

a_vars = []  # 1 if ordered ant from vender v in t
for t in range(S_T):
    a_vars.append([])
    for v in range(S_V):
        a_vars[t].append(case_2.addVar(lb = 0, vtype=GRB.BINARY,
                                       name=('a' + str(t+1) + ',' + str(v+1))))

p_vars = []  # 1 if product i be ordered by any method in t
for i in range(S_I):
    p_vars.append([])
    for t in range(S_T):
        p_vars[i].append(case_2.addVar(lb = 0, vtype=GRB.BINARY,
                                       name=('p' + str(i+1) + ',' + str(t+1))))

d_vars = []  # demand of product i in t as considering backorder
for i in range(S_I):
    d_vars.append([])
    for t in range(S_T):
        d_vars[i].append(case_2.addVar(lb = 0, vtype=GRB.CONTINUOUS,
                                       name=('d' + str(i+1) + ',' + str(t+1))))

## Objective Function

In [9]:
case_2.setObjective(quicksum(quicksum(Holding_cost[i] * u_vars[i][t] for t in range(S_T)) for i in range(S_I))
                    + quicksum(quicksum(quicksum((Shipping_cost[i][j] + Purchasing_cost[i]) * x_vars[i][j][t] for i in range(S_I)) + (Fixed_cost_shipping_method[j] * y_vars[j][t]) for j in range(S_J)) for t in range(S_T))
                    + quicksum(2750 * z_vars[t] for t in range(S_T))
                    + quicksum(quicksum((1-Backorder_percentage[i]) * Lost_sales_cost[i] * s_vars[i][t] for i in range(S_I)) for t in range(S_T))
                    + quicksum(quicksum(Backorder_percentage[i] * Backorder_cost[i] * s_vars[i][t] for i in range(S_I)) for t in range(S_T))
                    + quicksum(quicksum(Vendor_cost[v] * a_vars[t][v] for v in range(S_V)) for t in range(S_T)), GRB.MINIMIZE)

## Constrainsts

In [10]:
case_2.addConstrs((u_vars[i][0] - s_vars[i][0] == In_transit[i][0] - d_vars[i][0] for i in range(S_I)
                  ), '存貨水準定義（第一期）index = 0')

case_2.addConstrs((u_vars[i][1] - s_vars[i][1] == u_vars[i][0] + x_vars[i][0][0] + In_transit[i][1] - d_vars[i][1] for i in range(S_I)
                  ), '存貨水準定義（第二期）index = 1')

case_2.addConstrs((u_vars[i][2] - s_vars[i][2] == u_vars[i][1] + x_vars[i][0][1] + x_vars[i][1][0] + In_transit[i][2] - d_vars[i][2] for i in range(S_I)
                  ), '存貨水準定義（第三期）index = 2')

case_2.addConstrs((u_vars[i][t] - s_vars[i][t] == u_vars[i][t-1] + x_vars[i][0][t-1] + x_vars[i][1][t-2] + x_vars[i][2][t-3] + In_transit[i][t] - d_vars[i][t] for i in range(S_I) for t in range(3, S_T)
                  ), '存貨水準定義（從第四期開始）index = 3')

case_2.addConstrs((M * k_vars[i][t] >= s_vars[i][t] for i in range(S_I) for t in range(S_T)
                  ), '存貨水準與缺貨數量關係_1')
                  
case_2.addConstrs((M * (1 - k_vars[i][t]) >= u_vars[i][t] for i in range(S_I) for t in range(S_T)
                  ), '存貨水準與缺貨數量關係_2')

case_2.addConstrs((M * y_vars[j][t] >= quicksum(x_vars[i][j][t] for i in range(S_I)) for j in range(S_J) for t in range(S_T)
                  ), '有無使用此運送方式')

case_2.addConstrs((z_vars[t] >= (quicksum(Product_size[i] * x_vars[i][2][t] for i in range(S_I))) / 30 for t in range(S_T)
                  ), '貨櫃')

case_2.addConstrs((d_vars[i][0] >= Demand[i][0] for i in range(S_I)
                  ), '考量上期缺貨後的需求(第一期)')

case_2.addConstrs((d_vars[i][t] >= Demand[i][t] + (Backorder_percentage[i] * s_vars[i][t-1]) for i in range(S_I) for t in range(1, S_T)
                  ), '考量上期缺貨後的需求(第二期開始)')

for v in range(S_V):
    case_2.addConstrs((M * a_vars[t][v] >= quicksum((quicksum(x_vars[i][j][t] for j in range(S_J))) for i in range(Vendor_sepr_index[v], Vendor_sepr_index[v+1])) for t in range(S_T)
                      ), '當期是否有向該供應商訂購')

case_2.addConstrs((M * p_vars[i][t] >= quicksum(x_vars[i][j][t] for j in range(S_J)) for i in range(S_I) for t in range(S_T)
                  ), '當期是否有訂購該商品')
case_2.addConstrs((Bounds[i] * p_vars[i][t] <= quicksum(x_vars[i][j][t] for j in range(S_J)) for i in range(S_I) for t in range(S_T)
                  ), '如果有的話要訂超過最低限制的量')

case_2.addConstrs((1 >= p_vars[i][t] + p_vars[k][t] for i, k in Conflict for t in range(S_T)
                  ), '不能同時訂特定兩樣商品')

case_2.optimize()

IndexError: list index out of range

In [ ]:
# for var in case_2.getVars():
#     print(var.varName, '=', var.x)
print("objective value =", case_2.objVal)